# Sentiment analysis of IMDB movie reviews with various machine learning algorithms

### Exploratory data analysis
We will now explore the data and see what we are working with. This is useful to help us see what type of cleaning we need to do.`

#### Summary of the dataset
Once we have created the dataframe by reading the csv file, we can use describe(), df.shape, and df.head to form an overview of the data. We can also generate a word cloud to get an overview of the most occurring words.


In [13]:
# Description of the dataframe
imdb_data.describe()
#Lakshmipathi N 2020

,review,sentiment
count,50000,50000
unique,49581,2
top,love today show it wa a varieti and not sole c...,positive
freq,5,25000


We can also print out the shape of our dataset to see the number of rows and columns.

In [4]:
# Shape of the dataframe (rows, columns)
print(imdb_data.shape)
#Lakshmipathi N 2020

(50000, 2)


And we can use the head() function to see the first 10 rows.

In [5]:
# View the top 10 rows of the dataset
imdb_data.head(10)
#Lakshmipathi N 2020

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


Note the "</ br>" text - this is probably something we want to remove. We can use a wordcloud to help us see some other potential problems too.

In [6]:
# Concatenate the text from the review column into a single string.
wordcloud_text = ' '.join(df['review'])

# Generate the wordcloud
wordcloud = WordCloud(width=800, height=400).generate(wordcloud_text)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

# Adapted from Valkov 2019

NameError: name 'df' is not defined

Looks good, other than the "br" which we'll have to deal with.

#### Sentiment distribution
We want to ensure that we have a balanced dataset to avoid introducing bias into our model.


In [7]:
# Draw a bar chart to show the distribution of the data in the sentiment column.
axis = df['sentiment'].value_counts().sort_index()\
    .plot(kind="bar",
          title="Distribution of sentiment tags",
          figsize=(5,5))
axis.set_xlabel('Sentiments')
plt.show()
# New Code

NameError: name 'df' is not defined

Looks like the dataset is balanced. Great!

### Preprocessing
In this section we'll clean up the data. We'll be following Lakshmipathi N's approach to preprocessing for now and then introduce some additional steps later.

According to Valkov (2019) we should be aware that real world text data can be messy. When working with this dataset, they recommend removing HTML tags, punctuation and other special characters, and any excessive spaces.

We'll create some functions to do this cleaning for us and then apply them to the IMDB data.

#### Removing HTML and other junk text
Removing HTML and other junk text like the "<br>" we noticed in the word cloud.

In [ ]:
#Function to strip HTML tags
def strip_html(text):
    # Check if the input looks like a file name
    if os.path.isfile(text):
        with open(text, 'r') as file:
            soup = BeautifulSoup(file, "html.parser")
    else:
        soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

# Function to remove square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

# Function that combines the above to functions to clean up junk text
def remove_junk_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

# Apply the remove_junk_text function to the IMDB dataset
imdb_data['review'] = imdb_data['review'].apply(remove_junk_text)

#Adapted code from Lakshmipathi N 2020 with some changes

In [ ]:
# Check if the "<br>" tags we saw before has been removed.
imdb_data.head(10)

#### Removing special characters
Removing punctuation and other characters that are not letters or digits.

In [10]:
# Function to remove special characters
def remove_special_characters(text, remove_digits=True):
    pattern = r'[^a-zA-z0-9\s]'
    text = re.sub(pattern, '', text)
    return text

# Apply the remove_special_characters function to the dataset review  column
imdb_data['review'] = imdb_data['review'].apply(remove_special_characters)

#Lakshmipathi N 2020

#### Removing excessive spaces
Removing extra whitespace other than spaces between words.

In [11]:
# The function to remove white spaces
def remove_excessive_spaces(text):
    cleaned_text = re.sub('\s+', ' ', text)
    return cleaned_text.strip()

# Apply the function to the review column of the IMDB data
imdb_data['review'] = imdb_data['review'].apply(remove_excessive_spaces)

# New Code

#### Text stemming
Here we will be stemming the text using the Porter stemming algorithm and applying it to the 'review' column of the imdb_data dataframe.
By applying stemming, the code reduces words to their base or root form, which can help in text analysis tasks such as text classification, information retrieval, or topic modeling. Stemming aims to normalize words and treat different variations of the same word as the same base word.

In [ ]:
#Stemming the text with the Porter stemming algorithm from NLTK
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(simple_stemmer)

# Check the result of the stemming process
imdb_data.head(5)

#Lakshmipathi N 2020

### Removing stopwords
By removing stopwords we try to eliminate common words that often do not contribute significant meaning to the text analysis tasks. This can help improve the accuracy and efficiency of NLP applications such as sentiment analysis.

First, we set the stopwords list to English, and then let's just check the list of words that will be excluded in the next step.

In [9]:
# Setting English stopwords
stop=set(stopwords.words('english'))
print(stop)

#Lakshmipathi N 2020

{'but', 'our', 'between', 'why', 'up', "doesn't", 'when', 'there', 'what', 'have', 'which', 'are', 'only', 'him', 'further', "wasn't", 'just', 'under', 'for', "should've", 'in', 'herself', 'ain', 'then', 'whom', 'because', 'by', 'other', 'been', 'were', "hasn't", 'you', 'was', 'with', 'so', 'or', 'doing', 'her', 'we', 'all', 'is', 'any', 'can', 'those', 'am', 'nor', "couldn't", 'its', 'didn', 'should', 'his', 'did', 'i', 'me', 'again', 'as', "it's", 'this', 'of', 'no', "needn't", 'most', 'on', 'she', 'while', "aren't", 'few', 'own', 'more', 'about', 'against', 'shan', 'into', 'to', 'll', 'how', "won't", 'such', 'had', 'the', "isn't", "you've", 'both', 'very', 'where', 'their', 'themselves', 'ours', "haven't", 'my', 'over', "wouldn't", 'yourself', 'm', 'down', 'off', 'same', 'a', "mightn't", 'wasn', 'has', 'yourselves', 'ourselves', 'aren', 'wouldn', "shan't", 'they', 'hers', 'being', 's', 'haven', 'here', 'doesn', 'be', 'too', 'will', 'mustn', 'do', 'them', 'at', 'y', 'theirs', 'mightn

In [18]:
#Initialise a tokenizer
tokenizer=ToktokTokenizer()

# Function to remove the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stop]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stop]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(remove_stopwords)

#Lakshmipathi N 2020

# Test an example
review = "This is an example review with stopwords."
filtered_review = remove_stopwords(review)
print(filtered_review)
#New Code

example review stopwords .


### Feature extraction
In this section we'll do some work to represent the text data numerically by extracting relevant features. We'll use the Bag-of-Words model, where each review is represented as a vector of word frequencies or presence/absence indicators. This will pair well with the Naive Bayes model we'll be using later.

Let's use the CountVectorizer from the scikit-learn library to perform feature extraction with a bag-of-words approach.

In [19]:
# Use the CountVectorizer for bag of words
cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))

# Transform reviews
cv_reviews=cv.fit_transform(imdb_data.review)

# Check results
print('Bag of Words - cv_reviews:',cv_reviews.shape)

#Adapted code from Lakshmipathi N 2020 with some changes

Bag of Words - cv_reviews: (50000, 7914506)


We also need to convert the sentiment data into binary form. This is because when optimizing for sentiment analysis, occurrence of a word matters more than the frequency of the word.


Let's use LabelBinarizer from scikit-learn to give each sentiment value (which is "positive" and "negative" at the moment) a binary value.

In [20]:
# Label the sentiment data
lb=LabelBinarizer()

# Transform the sentiment data
sentiment_data=lb.fit_transform(imdb_data.sentiment)

# Check the results
print(sentiment_data.shape)

#Adapted code from Lakshmipathi N 2020 with some changes

(50000, 1)


### Testing and training data split
It's common to split a dataset into two separate sets to use for training and testing. As we can infer from the names, one set, usually larger, is used to train a model, while the other set is used to test the model on to assess the performance.

While it seems to work, the approach to splitting testing and training data used by Lakshmipathi N 2020 is somewhat cumbersome.

Let's use the train_test_split function from SKlearn to do this for us instead.

In [24]:
# Use sklearn to split the dataset into training and testing datasets
x = cv_reviews #input features
y = sentiment_data #labels
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

#New Code

## Classification approach

In this section we'll fit the data to a Naive Bayes model. We'll compare the results of this model with the baseline we got from the Lakshmipathi N notebook. We discussed this in the previous section.

#### Multinomial Naive Bayes classification
* Features are generated from a multinomial distribution by observing counts across categories.
* Data must be converted from strings to numbers. We can use TfidfVectorizer, or a bag-of-words approach with the CountVectorizer like we did earlier.

#### Train the model

In [47]:
# Train the multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
import numpy as np

mnb_model = MultinomialNB()

# Reshape train_sentiments into a 1-dimensional array
y_train_1d = np.ravel(y_train)
y_test_1d = np.ravel(y_test)

# Fit the bag of words data using the training reviews and 1-dimensional train_sentiments
mnb_model_bagofwords = mnb_model.fit(x_train, y_train_1d)
print(mnb_model_bagofwords)

#Adapted code from Lakshmipathi N 2020

MultinomialNB()


#### Using the model on test data

In [48]:
# Predict the model
mnb_model_bagofwords_predict = mnb_model_bagofwords.predict(x_test)


#### Model accuracy

In [49]:
#Accuracy score for bag of words

mnb_model_bagofwords_score = accuracy_score(y_test_1d,mnb_model_bagofwords_predict)
print("mnb_model_bagofwords_score :",mnb_model_bagofwords_score)

mnb_model_bagofwords_score : 0.4954


#### Classification report for benchmark

In [51]:
# Compile a classification report
from sklearn.metrics import classification_report
mnb_model_bagofwords_report = classification_report(y_test_1d, mnb_model_bagofwords_predict, target_names = ['Positive', 'Negative'])
print(mnb_model_bagofwords_report)

              precision    recall  f1-score   support

    Positive       0.50      0.97      0.66      5044
    Negative       0.23      0.01      0.01      4956

    accuracy                           0.50     10000
   macro avg       0.36      0.49      0.34     10000
weighted avg       0.36      0.50      0.34     10000



So for our first round, we got a accuracy score of 0.4954, which is much lower than the baseline of 0.751.

#### Iterating
Lets make some changes and see if they have an impact on our performance.

For the MNB model, the preprocessing techniques that was used for the baseline was:
* Removed HTML tags
* Remove special characters
* Remove spaces
* Stem the text
* Remove stopwords

Tinkering with the data
Let's rewrite all of the processing functions from before into a single function which does the following:
* Remove HTML tags and junk text
* Remove extra spaces
* Convert all words to lowercase (so ‘good’ and ‘Good’ aren’t counted is two different words)
* Keep special characters like punctuation to make handling negation easier


In [10]:
#New dataframe
import pandas as pd
movie_data = pd.read_csv('IMDB_dataset.csv')

In [11]:
#Preprocessing function

def preprocess(text):
    # Check if the input looks like a file name
    if os.path.isfile(text):
        with open(text, 'r') as file:
            soup = BeautifulSoup(file, "html.parser")
    else:
        soup = BeautifulSoup(text, "html.parser")

    cleaned_text = soup.get_text()

    # Replace [] with nothing
    cleaned_text = re.sub(r'\[[^]]*\]', '', cleaned_text)

    # Remove excessive white spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()

    # Convert to lowercase
    cleaned_text = cleaned_text.lower()

    # Preserve negation
    cleaned_text = re.sub(r'\b(n\'t|not|no|never)\s+(\w+)', r'not_\1 \2', cleaned_text)

    return cleaned_text

#Adapted code from Lakshmipathi N 2020 with substantial changes

In [12]:
# Apply preprossing to the dataframe
movie_data['review'] = movie_data['review'].apply(preprocess)

/var/folders/fp/sfmgsyr555b7vfy1lp18nrjw0000gn/T/ipykernel_65281/3295716233.py:9: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")



In the preprocessing function above, we have avoided removing punctuation and also tried to preserve negation. There are some further things we will do differently:
* Avoid stemming the words
* Avoid removing stopwords (According to Jurafsky and Martin (2023) removing stopwords doesn’t improve the performance of text classifications applications.)

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Initialize the CountVectorizer
vectorizer = CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))

# Fit the vectorizer on the preprocessed text data
X = vectorizer.fit_transform(movie_data.review)

# Convert the feature matrix to an array
X = X.toarray()

# Get the feature names from the vocabulary
feature_names = vectorizer.get_feature_names()

In [1]:
# Print the shape of the feature matrix
print("Shape of feature matrix:", X.shape)

NameError: name 'X' is not defined

## III Conclusions
tbc